# Quickstart

## Using the Block Model API Client

Using the notebook utilities provided by `evo-sdk-common`, we can easily interact with the Block Model Service in a Jupyter notebook environment.

In [ ]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="your-client-id",
    cache_location="./notebook-data",
).login()

## BlockModelAPIClient

The `BlockModelAPIClient` wraps endpoint functionality to provide a cohesive interface to the underlying API implementation.

In [ ]:
from evo.blockmodels import BlockModelAPIClient

service_client = BlockModelAPIClient.from_context(manager)
service_health = await service_client.get_service_health()

print(f"Block Model Service is {service_health.status.name.lower()}")

### List Block Models

In [ ]:
from evo.blockmodels.data import SubBlockedGridDefinition

# List the first page
models = await service_client.list_block_models()
print(f"Found {len(models)} models. First model id: {models[0].id if len(models) > 0 else None}")

# Fetch all models across pages with a page size of 50
all_models = await service_client.list_all_block_models(page_limit=5)
print(f"Total models returned: {len(all_models)}")

# Inspect a model
if len(all_models) > 0:
    block_model = all_models[0]
    print(f"First model name: {block_model.name}")
    if isinstance(block_model.grid_definition, SubBlockedGridDefinition):
        print(f"First model grid definition [nx, ny, nz]: {block_model.grid_definition.n_parent_blocks}")
    else:
        print(f"First model grid definition [nx, ny, nz]: {block_model.grid_definition.n_blocks}")

### List Block Model Versions

In [ ]:
# List the first page of versions
versions = await service_client.list_versions(bm_id=block_model.id)
print(f"Found {len(versions)} versions")

# List all versions of a block model
# Versions are ordered from newest to oldest
all_versions = await service_client.list_all_versions(bm_id=block_model.id)

print(f"Total versions: {len(all_versions)}")
# Inspect a version
if len(all_versions) > 0:
    version = all_versions[0]
    print(f"Latest Version {version.version_id}: {version.version_uuid} (created: {version.created_at})")

### Create Block Model

In [ ]:
import pyarrow

from evo.blockmodels.data import RegularGridDefinition
from evo.blockmodels.endpoints.models import RotationAxis

block_grid = RegularGridDefinition(
    model_origin=[0, 0, 0], rotations=[(RotationAxis.x, 20)], n_blocks=[10, 10, 10], block_size=[1, 1, 1]
)
initial_data = pyarrow.table(
    {"i": [1, 2, 3], "j": [4, 5, 6], "k": [7, 8, 9], "column_one": ["A", "D", "E"], "column_two": [1.5, 1.3, 1.2]},
    schema=pyarrow.schema(
        {
            "i": pyarrow.uint32(),
            "j": pyarrow.uint32(),
            "k": pyarrow.uint32(),
            "column_one": pyarrow.string(),
            "column_two": pyarrow.float32(),
        }
    ),
)
block_model, version = await service_client.create_block_model(
    name="My Block Model",
    description="My Block Model",
    grid_definition=block_grid,
    object_path="your/path",
    coordinate_reference_system="EPSG:3395",
    size_unit_id="m",
    initial_data=initial_data,
)

### Add New Columns

In [ ]:
new_cols = pyarrow.table(
    {"i": [1, 1], "j": [4, 4], "k": [6, 7], "column_three": ["C", "D"], "column_four": [4.5, 5.3]},
    schema=pyarrow.schema(
        {
            "i": pyarrow.uint32(),
            "j": pyarrow.uint32(),
            "k": pyarrow.uint32(),
            "column_three": pyarrow.string(),
            "column_four": pyarrow.float32(),
        }
    ),
)

version = await service_client.add_new_columns(
    bm_id=block_model.id,
    data=new_cols,
    units={"column_four": "g/t"},
)

### Query Block Model as Table

In [ ]:
from evo.blockmodels.endpoints.models import BBox, IntRange

# Select a bounding box to query
# This example selects blocks where: 1 <= i <= 2
bounding_box = BBox(
    i_minmax=IntRange(min=1, max=2),
    j_minmax=IntRange(min=1, max=9),
    k_minmax=IntRange(min=1, max=9),
)

# Table will be a Pyarrow Table
await service_client.query_block_model_as_table(
    bm_id=block_model.id,
    columns=["column_one", "column_four"],
    bbox=bounding_box,
    version_uuid=version.version_uuid,
)

### Update column units

In [ ]:
# Update units for an existing column and add a comment describing the change
# This example updates `column_two` to use percent (mass) and records a comment on the new version.
version = await service_client.update_column_metadata(
    bm_id=block_model.id,
    column_updates={"column_two": "%[mass]"},
    comment="Set column_two to percent mass for downstream analysis",
)

print(f"Updated version: id={version.version_id}, uuid={version.version_uuid}, comment={version.comment}")

### Rename Block Model Columns

In [ ]:
# Rename existing columns in the block model
# This example renames columns to more descriptive names
version = await service_client.rename_block_model_columns(
    bm_id=block_model.id,
    column_renames={"column_one": "geology_type", "column_two": "assay_value"},
    comment="Renamed columns for clarity and consistency",
)

print(f"Renamed version: id={version.version_id}, uuid={version.version_uuid}, comment={version.comment}")

### Delete Block Model Columns

In [ ]:
# Delete columns from a block model
# This example deletes two columns and records a comment on the new version.
version_after_delete = await service_client.delete_block_model_columns(
    bm_id=block_model.id,
    column_titles=["column_three", "column_four"],
    comment="Remove temporary columns",
)

print(
    f"After delete: id={version_after_delete.version_id}, uuid={version_after_delete.version_uuid}, comment={version_after_delete.comment}"
)

### Create Sub-blocked Model

In addition to supporting regular block models, the Block Model API also supports sub-blocked models. For information on sub-blocked models, see the [Block Model API documentation](https://developer.seequent.com/docs/guides/blockmodel/geometry/sub-blocking).

In this example, we will create a fully sub-blocked model, which means each parent block can either be undivided or divided into a fixed number of sub-blocks along each axis. 

In [ ]:
from evo.blockmodels.data import FullySubBlockedGridDefinition

# Similar to regular block models, sub-blocked models can be created with the create_block_model method, but uses a different
# grid definition.
block_grid = FullySubBlockedGridDefinition(
    model_origin=[0, 0, 0],
    rotations=[],
    n_parent_blocks=[10, 10, 10],
    parent_block_size=[2, 1, 1],
    n_subblocks_per_parent=[5, 1, 2],
)

# As an alternative to specifying the index columns, data tables for sub-blocked models can also be defined by columns defining the block centroids (x, y, z) and sizes (dx, dy, dz).
initial_data = pyarrow.table(
    {
        "x": [1.0, 2.6, 3.0],
        "y": [1.5, 2.5, 2.5],
        "z": [4.5, 2.25, 2.25],
        "dx": [2.0, 0.4, 0.4],
        "dy": [1.0, 1.0, 1.0],
        "dz": [1.0, 0.5, 0.5],
        "column_one": ["A", "D", "E"],
        "column_two": [1.5, 1.3, 1.2],
    },
    schema=pyarrow.schema(
        {
            "x": pyarrow.float64(),
            "y": pyarrow.float64(),
            "z": pyarrow.float64(),
            "dx": pyarrow.float64(),
            "dy": pyarrow.float64(),
            "dz": pyarrow.float64(),
            "column_one": pyarrow.string(),
            "column_two": pyarrow.float32(),
        }
    ),
)
block_model, version = await service_client.create_block_model(
    name="My Sub-Blocked Model",
    description="My Sub-Blocked Model",
    grid_definition=block_grid,
    object_path="your/path",
    coordinate_reference_system="EPSG:3395",
    size_unit_id="m",
    initial_data=initial_data,
)

### Add New Sub-blocked Columns

In [ ]:
# For the add_new_subblocked_columns method, the sub-blocks within the data table must match existing sub-blocks defined in the model.
new_cols = pyarrow.table(
    {
        "i": [0, 1],
        "j": [1, 2],
        "k": [4, 2],
        # The sidx of 0 means that the sub-blocks is the full undivided parent block.
        # The sidx of 3 means that the sub-block is the third sub-block within the divided grid, which maps to the (1, 0, 0) position in the sub-block grid.
        # For more explanation of sidx, see the Block Model API documentation here: https://developer.seequent.com/docs/guides/blockmodel/geometry/sub-blocking#fully-sub-blocked-models-1
        "sidx": [0, 3],
        "column_three": ["C", "D"],
        "column_four": [4.5, 5.3],
    },
    schema=pyarrow.schema(
        {
            "i": pyarrow.uint32(),
            "j": pyarrow.uint32(),
            "k": pyarrow.uint32(),
            "sidx": pyarrow.uint32(),
            "column_three": pyarrow.string(),
            "column_four": pyarrow.float32(),
        }
    ),
)
version = await service_client.add_new_subblocked_columns(
    bm_id=block_model.id,
    data=new_cols,
    units={"column_four": "g/t"},
)

### Query Sub-blocked Model as Table

In [ ]:
# This behaves the same as querying a regular block model, but it also returns additional columns like the sub-block sizes (dx, dy, dz).
await service_client.query_block_model_as_table(
    bm_id=block_model.id,
    columns=["column_one", "column_four"],
    version_uuid=version.version_uuid,
)

### Update Geometry of Sub-blocked Model

To update the geometry of a sub-blocked model, we can use the `update_subblocked_columns` method, with the `geometry_change` parameter set to `True`. This requires all existing columns to be either updated or deleted, as part of this operation.

In [ ]:
new_cols = pyarrow.table(
    {
        "i": [0, 1],
        "j": [1, 2],
        "k": [4, 2],
        "sidx": [1, 0],
        "column_two": [11.5, 12.3],
        "column_three": ["X", "Y"],
        "column_five": [14, -1],
    },
    schema=pyarrow.schema(
        {
            "i": pyarrow.uint32(),
            "j": pyarrow.uint32(),
            "k": pyarrow.uint32(),
            "sidx": pyarrow.uint32(),
            "column_two": pyarrow.float32(),
            "column_three": pyarrow.string(),
            "column_five": pyarrow.int32(),
        }
    ),
)

version = await service_client.update_subblocked_columns(
    bm_id=block_model.id,
    data=new_cols,
    update_columns={"column_two", "column_three"},
    delete_columns={"column_one", "column_four"},
    new_columns=["column_five"],
    geometry_change=True,
)
await service_client.query_block_model_as_table(bm_id=block_model.id, version_uuid=version.version_uuid, columns=["*"])